In [4]:
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [6]:
#savant_df2016 = statcast(start_dt = "2016-03-28", end_dt = "2016-11-15")

In [7]:
#savant_df2017 = statcast(start_dt = "2017-03-28", end_dt = "2017-11-15")

In [8]:
#savant_df2018 = statcast(start_dt = "2018-03-28", end_dt = "2018-11-15")

In [9]:
#savant_df2021 = statcast(start_dt = "2021-03-28", end_dt = "2021-11-15")

In [10]:
#savant_df2022 = statcast(start_dt = "2022-03-28", end_dt = "2022-11-15")

In [11]:
#expected = pd.concat([savant_df2016, savant_df2017, savant_df2018, savant_df2021, savant_df2022], axis = 0)

In [12]:
expected = pd.read_csv("Expected.csv")

#pd.set_option('display.max_columns', None)

expected

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_42062/1507044722.py:1: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  expected = pd.read_csv("Expected.csv")


,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
1,CU,2016-11-02,74.3,3.04,6.51,Mike Montgomery,492841,543557,NaN,called_strike,...,8.0,Standard,Standard,318.0,0.000,-0.040,-15.48,-8.16,8.16,3.04
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
3,FF,2016-11-02,95.6,-1.28,6.51,Carl Edwards Jr.,434658,605218,NaN,ball,...,8.0,Standard,Standard,191.0,0.002,0.031,17.64,-2.16,2.16,1.28
4,FF,2016-11-02,94.8,-1.13,6.30,Carl Edwards Jr.,446386,605218,walk,ball,...,8.0,Standard,Standard,185.0,0.030,0.067,14.52,2.76,2.76,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658661,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,621563,571578,NaN,ball,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3658662,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,621563,571578,NaN,ball,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,0.0,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN


In [13]:
# Converts the Induced Vertical Break and Horizontal Break to inches from feet
expected['IVB'] = expected['pfx_z'] * 12
expected['HB'] = expected['pfx_x'] * 12
expected['ABS_HB'] = np.abs(expected["HB"])
expected['ABS_RelSide'] = np.abs(expected["release_pos_x"])

expected = expected.rename(columns={'player_name': 'Name', "pitch_type" : "Pitch Type", "release_pos_x" : "RelSide", "release_pos_z" : "RelHeight"})
expected

,Pitch Type,game_date,release_speed,RelSide,RelHeight,Name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
1,CU,2016-11-02,74.3,3.04,6.51,Mike Montgomery,492841,543557,NaN,called_strike,...,8.0,Standard,Standard,318.0,0.000,-0.040,-15.48,-8.16,8.16,3.04
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
3,FF,2016-11-02,95.6,-1.28,6.51,Carl Edwards Jr.,434658,605218,NaN,ball,...,8.0,Standard,Standard,191.0,0.002,0.031,17.64,-2.16,2.16,1.28
4,FF,2016-11-02,94.8,-1.13,6.30,Carl Edwards Jr.,446386,605218,walk,ball,...,8.0,Standard,Standard,185.0,0.030,0.067,14.52,2.76,2.76,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658661,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,621563,571578,NaN,ball,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3658662,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,621563,571578,NaN,ball,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,0.0,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN


In [14]:
df_swing = expected[expected['description'].isin(['swinging_strike', 'swinging_strike_blocked', 'hit_into_play', 'foul'])]
df_swing

,Pitch Type,game_date,release_speed,RelSide,RelHeight,Name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
9,CU,2016-11-02,81.3,-0.98,6.39,Carl Edwards Jr.,608070,605218,field_out,hit_into_play,...,8.0,Standard,Standard,44.0,-0.029,-0.144,-11.88,13.92,13.92,0.98
12,CU,2016-11-02,82.4,-1.17,6.30,Carl Edwards Jr.,608070,605218,NaN,foul,...,8.0,Standard,Standard,51.0,0.000,-0.037,-12.72,12.96,12.96,1.17
14,FF,2016-11-02,95.2,-1.16,6.37,Carl Edwards Jr.,435063,605218,strikeout,swinging_strike,...,8.0,Standard,Standard,165.0,-0.044,-0.208,14.04,5.28,5.28,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658657,NaN,2022-03-28,NaN,NaN,NaN,Edward Cabrera,514917,665795,field_out,hit_into_play,...,0.0,NaN,NaN,NaN,-0.021,NaN,NaN,NaN,NaN,NaN
3658658,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,643265,571578,grounded_into_double_play,hit_into_play,...,0.0,NaN,NaN,NaN,0.050,NaN,NaN,NaN,NaN,NaN
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,0.0,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN


In [15]:
df_swing['Whiff'] = df_swing['description'].apply(lambda x: 1 if x == 'swinging_strike' else 0)
df_swing

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_42062/3217903474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swing['Whiff'] = df_swing['description'].apply(lambda x: 1 if x == 'swinging_strike' else 0)


,Pitch Type,game_date,release_speed,RelSide,RelHeight,Name,batter,pitcher,events,description,...,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide,Whiff
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93,0
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29,0
9,CU,2016-11-02,81.3,-0.98,6.39,Carl Edwards Jr.,608070,605218,field_out,hit_into_play,...,Standard,Standard,44.0,-0.029,-0.144,-11.88,13.92,13.92,0.98,0
12,CU,2016-11-02,82.4,-1.17,6.30,Carl Edwards Jr.,608070,605218,NaN,foul,...,Standard,Standard,51.0,0.000,-0.037,-12.72,12.96,12.96,1.17,0
14,FF,2016-11-02,95.2,-1.16,6.37,Carl Edwards Jr.,435063,605218,strikeout,swinging_strike,...,Standard,Standard,165.0,-0.044,-0.208,14.04,5.28,5.28,1.16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658657,NaN,2022-03-28,NaN,NaN,NaN,Edward Cabrera,514917,665795,field_out,hit_into_play,...,NaN,NaN,NaN,-0.021,NaN,NaN,NaN,NaN,NaN,0
3658658,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,643265,571578,grounded_into_double_play,hit_into_play,...,NaN,NaN,NaN,0.050,NaN,NaN,NaN,NaN,NaN,0
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN,1
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,1


In [16]:
df_swing = df_swing[["Name", "Pitch Type", "release_speed", "release_spin_rate", "IVB", "ABS_HB", "RelHeight", "ABS_RelSide", "release_extension", "Whiff", "RelSide", "HB", "plate_x", "plate_z"]].dropna()
df_swing

,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,Whiff,RelSide,HB,plate_x,plate_z
0,Mike Montgomery,CU,76.6,2324.0,-15.84,9.36,6.59,2.93,6.2,0,2.93,-9.36,0.75,2.85
2,Carl Edwards Jr.,FF,94.9,2636.0,17.52,0.48,6.46,1.29,6.8,0,-1.29,0.48,0.04,1.56
9,Carl Edwards Jr.,CU,81.3,2468.0,-11.88,13.92,6.39,0.98,6.4,0,-0.98,13.92,-1.20,2.10
12,Carl Edwards Jr.,CU,82.4,2695.0,-12.72,12.96,6.30,1.17,5.4,0,-1.17,12.96,0.47,1.54
14,Carl Edwards Jr.,FF,95.2,2703.0,14.04,5.28,6.37,1.16,6.9,1,-1.16,5.28,1.71,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658515,Adam Wainwright,FC,84.0,2349.0,8.88,6.24,6.32,1.26,6.4,0,-1.26,6.24,0.16,2.06
3658518,Adam Wainwright,SI,88.4,2063.0,12.84,14.04,6.21,1.13,6.5,0,-1.13,-14.04,0.48,2.51
3658519,Adam Wainwright,CH,81.9,1610.0,5.04,10.56,6.26,1.34,6.2,1,-1.34,-10.56,-0.94,1.99
3658520,Adam Wainwright,SI,87.7,2088.0,9.60,11.52,6.26,1.29,6.4,0,-1.29,-11.52,0.23,2.30


In [17]:
df_sinker = df_swing[(df_swing["Pitch Type"] == "SI")]
df_cutter = df_swing[df_swing["Pitch Type"] == "FC"]
df_fastball = df_swing[df_swing["Pitch Type"] == "FF"]
df_changeup = df_swing[df_swing["Pitch Type"] == "CH"]
df_splitter = df_swing[(df_swing["Pitch Type"] == "FS") | (df_swing["Pitch Type"] == "FO")]
df_curveball = df_swing[(df_swing["Pitch Type"] == "CU") | (df_swing["Pitch Type"] == "KC")]
df_slider = df_swing[(df_swing["Pitch Type"] == "SL") | (df_swing["Pitch Type"] == "SV")]
df_sweeper = df_swing[df_swing["Pitch Type"] == "ST"]

## Sinker

In [18]:
SIfeature = df_sinker.drop(["Name", "Pitch Type", "Whiff", "RelSide","HB", "plate_x", "plate_z"], axis=1)
SIlabel = df_sinker['Whiff']

SIrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(SIfeature, SIlabel)

## Cutter

In [19]:
CTfeature = df_cutter.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
CTlabel = df_cutter['Whiff']

CTrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(CTfeature, CTlabel)

## 4-Seam

In [20]:
FBfeature = df_fastball.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
FBlabel = df_fastball['Whiff']

FBrf = RandomForestRegressor(n_estimators = 30, random_state = 42).fit(FBfeature, FBlabel)

## Changeup

In [21]:
CHfeature = df_changeup.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
CHlabel = df_changeup['Whiff']

CHrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(CHfeature, CHlabel)

## Splitter

In [22]:
SPfeature = df_splitter.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
SPlabel = df_splitter['Whiff']

SPrf = RandomForestRegressor(n_estimators = 10, random_state = 42).fit(SPfeature, SPlabel)

## Curveball

In [23]:
CUfeature = df_curveball.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
CUlabel = df_curveball['Whiff']

CUrf = RandomForestRegressor(n_estimators = 20, random_state = 42).fit(CUfeature, CUlabel)

## Slider

In [24]:
SLfeature = df_slider.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
SLlabel = df_slider['Whiff']

SLrf = RandomForestRegressor(n_estimators = 25, random_state = 42).fit(SLfeature, SLlabel)

## Sweeper

In [25]:
STfeature = df_sweeper.drop(["Name", "Pitch Type", "Whiff", "RelSide", "HB", "plate_x", "plate_z"], axis=1)
STlabel = df_sweeper['Whiff']

STrf = RandomForestRegressor(n_estimators = 10, random_state = 42).fit(STfeature, STlabel)

## 2023 DF

In [26]:
savant_df = statcast(start_dt = "2023-03-30", end_dt = "2023-10-02")

savant_df['IVB'] = savant_df['pfx_z'] * 12
savant_df['HB'] = savant_df['pfx_x'] * 12
savant_df['ABS_HB'] = np.abs(savant_df["HB"])
savant_df['ABS_RelSide'] = np.abs(savant_df["release_pos_x"])

savant_df = savant_df.rename(columns={'player_name': 'Name', "pitch_type" : "Pitch Type", "release_pos_x" : "RelSide", "release_pos_z" : "RelHeight"})

def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

savant_df['Name'] = savant_df["Name"].apply(flip_names)

savant_df['pitcher_team'] = savant_df.apply(lambda row: row['home_team'] if row['inning_topbot'] == 'Top' else row['away_team'], axis=1)

nl_teams = ["ATL", "NYM", "MIA", "PHI", "WSH", "CHC", "CIN", "MIL", "PIT", "STL", "LAD", "ARI", "SFG", "SDP", "COL"]
savant_df["league"] = np.where(savant_df["pitcher_team"].isin(nl_teams), "NL", "AL")


savant_df = savant_df[["league", "pitcher_team", "Name", "Pitch Type", "release_speed", "release_spin_rate", "IVB", "ABS_HB", "RelHeight", "ABS_RelSide", "release_extension", "RelSide", "HB", "plate_x", "plate_z"]].dropna()
savant_df = savant_df.reset_index(drop = True).dropna()

savant_df

This is a large query, it may take a moment to complete


100%|██████████| 187/187 [02:06<00:00,  1.47it/s]


,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,RelSide,HB,plate_x,plate_z
0,AL,BOS,Nick Robertson,CH,89.0,1703,3.96,18.36,5.59,2.8,7.4,-2.8,-18.36,0.33,2.01
1,AL,BOS,Nick Robertson,FF,96.9,2153,16.32,9.12,5.9,2.4,7.4,-2.4,-9.12,0.09,2.71
2,AL,BOS,Nick Robertson,CH,90.0,1698,4.32,19.8,5.56,2.93,7.4,-2.93,-19.8,-0.24,0.53
3,AL,BOS,Nick Robertson,SL,82.2,2786,3.36,17.16,5.55,3.09,6.9,-3.09,17.16,0.81,0.49
4,AL,BOS,Nick Robertson,CH,89.2,1589,3.72,17.88,5.58,2.87,7.2,-2.87,-17.88,-0.78,2.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713249,AL,SEA,Luis Castillo,SI,95.3,2120,5.52,20.28,5.24,3.11,5.6,-3.11,-20.28,-0.99,2.71
713250,AL,SEA,Luis Castillo,FF,94.2,2284,13.92,15.36,5.29,3.16,5.6,-3.16,-15.36,-0.73,2.89
713251,AL,SEA,Luis Castillo,FF,94.8,2122,14.28,9.6,5.3,3.12,5.4,-3.12,-9.6,1.6,2.42
713252,AL,SEA,Luis Castillo,SI,95.1,2126,4.92,20.52,5.22,3.2,5.5,-3.2,-20.52,-1.31,2.23


In [27]:
player = savant_df[savant_df["Name"] == "Corbin Burnes"].reset_index(drop = True).dropna()
player

,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,RelSide,HB,plate_x,plate_z
0,NL,MIL,Corbin Burnes,FC,94.7,2664,9.12,7.56,5.62,0.92,6.5,-0.92,7.56,0.47,2.72
1,NL,MIL,Corbin Burnes,FC,95.4,2691,10.92,3.36,5.71,1.0,6.3,-1.0,3.36,0.19,4.74
2,NL,MIL,Corbin Burnes,SL,83.9,2934,3.12,20.28,5.47,1.07,6.6,-1.07,20.28,0.32,2.2
3,NL,MIL,Corbin Burnes,SL,88.0,2895,7.08,8.28,5.54,1.04,6.7,-1.04,8.28,-0.02,1.78
4,NL,MIL,Corbin Burnes,FC,94.4,2754,10.68,5.76,5.51,0.99,6.8,-0.99,5.76,0.2,1.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,NL,MIL,Corbin Burnes,SL,87.9,2991,0.84,6.12,5.87,0.93,6.5,-0.93,6.12,0.17,0.64
3052,NL,MIL,Corbin Burnes,FC,96.3,2717,17.16,3.96,6.05,0.79,6.3,-0.79,3.96,0.22,3.3
3053,NL,MIL,Corbin Burnes,SL,89.4,2865,3.0,5.76,5.95,0.79,6.4,-0.79,5.76,0.37,1.36
3054,NL,MIL,Corbin Burnes,FC,95.2,2617,12.12,1.68,5.9,0.87,6.3,-0.87,1.68,-0.53,1.61


In [28]:
team = savant_df[savant_df["pitcher_team"] == "ATL"].reset_index(drop = True).dropna()
team

,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,RelSide,HB,plate_x,plate_z
0,NL,ATL,Michael Tonkin,SL,82.0,2195,-5.52,2.52,5.4,3.83,6.3,-3.83,2.52,-0.45,1.18
1,NL,ATL,Michael Tonkin,SI,94.6,2454,14.16,9.12,5.45,3.63,6.2,-3.63,-9.12,0.9,2.96
2,NL,ATL,Michael Tonkin,SI,94.1,2495,10.92,9.24,5.36,3.8,6.3,-3.8,-9.24,-0.53,3.41
3,NL,ATL,Michael Tonkin,SL,84.0,2452,-2.16,3.36,5.38,3.73,6.2,-3.73,3.36,-1.12,1.71
4,NL,ATL,Michael Tonkin,SL,83.9,2353,0.24,3.36,5.46,3.65,6.2,-3.65,3.36,0.59,1.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23927,NL,ATL,Max Fried,CU,76.6,2692,-15.84,8.28,6.23,1.29,6.3,1.29,-8.28,-0.89,0.03
23928,NL,ATL,Max Fried,CU,75.6,2866,-16.44,5.64,6.19,1.23,6.5,1.23,-5.64,-0.64,0.77
23929,NL,ATL,Max Fried,CH,87.3,1701,5.16,17.76,5.88,1.51,6.4,1.51,17.76,0.29,2.63
23930,NL,ATL,Max Fried,CU,73.1,2612,-13.2,6.72,6.29,1.29,6.0,1.29,-6.72,-0.07,3.54


In [72]:
league = savant_df[savant_df["league"] == "AL"].reset_index(drop = True).dropna()
league

,league,pitcher_team,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,RelSide,HB,plate_x,plate_z
0,AL,BOS,Nick Robertson,CH,89.0,1703,3.96,18.36,5.59,2.8,7.4,-2.8,-18.36,0.33,2.01
1,AL,BOS,Nick Robertson,FF,96.9,2153,16.32,9.12,5.9,2.4,7.4,-2.4,-9.12,0.09,2.71
2,AL,BOS,Nick Robertson,CH,90.0,1698,4.32,19.8,5.56,2.93,7.4,-2.93,-19.8,-0.24,0.53
3,AL,BOS,Nick Robertson,SL,82.2,2786,3.36,17.16,5.55,3.09,6.9,-3.09,17.16,0.81,0.49
4,AL,BOS,Nick Robertson,CH,89.2,1589,3.72,17.88,5.58,2.87,7.2,-2.87,-17.88,-0.78,2.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427753,AL,SEA,Luis Castillo,SI,95.3,2120,5.52,20.28,5.24,3.11,5.6,-3.11,-20.28,-0.99,2.71
427754,AL,SEA,Luis Castillo,FF,94.2,2284,13.92,15.36,5.29,3.16,5.6,-3.16,-15.36,-0.73,2.89
427755,AL,SEA,Luis Castillo,FF,94.8,2122,14.28,9.6,5.3,3.12,5.4,-3.12,-9.6,1.6,2.42
427756,AL,SEA,Luis Castillo,SI,95.1,2126,4.92,20.52,5.22,3.2,5.5,-3.2,-20.52,-1.31,2.23


# For Players

In [30]:
stuff = player.drop(["league", "pitcher_team", "Name", "Pitch Type", "RelSide", "HB", "plate_x", "plate_z"], axis = 1)

In [31]:
for i in range(len(player)):
    pitch_type = player.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        player.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        player.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        player.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        player.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO"  or pitch_type == "FS":
        player.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        player.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        player.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        player.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [32]:
player_grouped = player.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    HB = ('HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2)

player_grouped

Pitches   Velo  Max_Velo  SpinRate    IVB     HB  \
Name          Pitch Type                                                     
Corbin Burnes CH              344  89.32      91.9   2048.71   7.07 -13.83   
              CU              526  79.86      83.7   2794.21 -10.94  10.35   
              FC             1693  94.43      97.3   2655.19  11.24   3.13   
              SI              235  95.31      97.8   2489.25  10.89 -10.68   
              SL              258  86.03      91.8   2876.97   1.56  12.45   

                          Release_Height  Extension   Stuff  
Name          Pitch Type                                     
Corbin Burnes CH                    5.69       6.62  126.78  
              CU                    6.07       6.53  131.05  
              FC                    5.91       6.57  120.88  
              SI                    5.88       6.62  157.61  
              SL                    5.79       6.56  137.94

# For Teams

In [33]:
stuff = team.drop(["league", "pitcher_team", "Name", "Pitch Type", "RelSide", "HB", "plate_x", "plate_z"], axis = 1)

In [34]:
for i in range(len(team)):
    pitch_type = team.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        team.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        team.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        team.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        team.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO" or pitch_type == "FS":
        team.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        team.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        team.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        team.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [35]:
team_grouped = team.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    HB = ('HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index()

team_grouped

,Name,Pitch Type,Pitches,Velo,Max_Velo,SpinRate,IVB,HB,Release_Height,Extension,Stuff
0,A.J. Minter,CH,144,86.68,89.2,1636.7,5.05,15.39,5.66,6.23,91.70
1,A.J. Minter,FC,408,89.45,92.2,2557.94,3.01,-5.44,5.8,6.28,155.44
2,A.J. Minter,FF,496,95.84,97.7,2426.04,16.7,6.51,5.79,6.38,137.95
3,AJ Smith-Shawver,CH,48,86.09,89.0,1354.75,9.64,-9.78,6.0,6.7,104.59
4,AJ Smith-Shawver,CU,55,76.79,80.3,2255.8,-13.02,8.31,6.23,6.53,104.90
...,...,...,...,...,...,...,...,...,...,...,...
117,Taylor Hearn,SL,3,88.33,89.6,2032.67,2.88,-3.72,6.61,6.27,91.73
118,Yonny Chirinos,FF,12,93.08,94.2,2188.83,13.9,-9.64,6.53,6.16,67.31
119,Yonny Chirinos,FS,151,85.72,88.1,1356.2,3.63,-6.06,6.81,6.35,115.05
120,Yonny Chirinos,SI,180,92.73,95.6,2037.03,6.77,-16.07,6.61,6.33,208.29


In [36]:
Sinker = team_grouped[team_grouped["Pitch Type"] == "SI"].sort_values(by='Stuff', ascending = False)

In [37]:
Cutter = team_grouped[team_grouped["Pitch Type"] == "FC"].sort_values(by='Stuff', ascending = False)

In [38]:
Fastball = team_grouped[team_grouped["Pitch Type"] == "FF"].sort_values(by='Stuff', ascending = False)

In [39]:
Changeup = team_grouped[team_grouped["Pitch Type"] == "CH"].sort_values(by='Stuff', ascending = False)

In [40]:
Splitter = team_grouped[team_grouped["Pitch Type"].isin(["SP", "FO", "FS"])].sort_values(by='Stuff', ascending = False)

In [41]:
Curveball = team_grouped[team_grouped["Pitch Type"].isin(["CU", "KC"])].sort_values(by='Stuff', ascending = False)

In [42]:
Slider = team_grouped[team_grouped["Pitch Type"].isin(["SL", "SV"])].sort_values(by='Stuff', ascending = False)

In [43]:
Sweeper = team_grouped[team_grouped["Pitch Type"] == "ST"].sort_values(by='Stuff', ascending = False)

In [44]:
overall = team.groupby(['Name',]).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    HB = ('HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index().sort_values(by='Stuff', ascending = False)

overall

,Name,Pitches,Velo,Max_Velo,SpinRate,IVB,HB,Release_Height,Extension,Stuff
8,Danny Young,135,83.63,90.4,2537.7,3.07,1.11,5.43,5.63,157.12
31,Yonny Chirinos,394,89.74,95.6,1803.25,5.83,-10.15,6.69,6.33,157.07
30,Taylor Hearn,24,95.3,97.3,2281.25,13.75,7.81,6.6,6.32,141.60
17,Joe Jiménez,936,91.53,97.5,2334.18,11.82,-5.14,5.83,7.11,139.84
29,Spencer Strider,3070,92.57,100.5,2348.54,12.18,-2.72,5.85,7.05,138.64
0,A.J. Minter,1048,92.1,97.7,2368.93,9.77,3.08,5.77,6.32,138.40
27,Pierce Johnson,341,88.33,97.4,2789.66,0.57,3.48,6.11,5.87,132.09
24,Michael Tonkin,1281,89.43,96.0,2290.42,5.44,-6.05,5.32,6.26,127.75
6,Charlie Morton,2840,88.05,98.0,2629.09,1.02,-1.37,5.37,6.15,125.59
18,Kirby Yates,1058,90.02,95.9,1882.28,7.09,-12.73,5.32,6.15,122.91


# For League

In [73]:
stuff = league.drop(["league", "pitcher_team", "Name", "Pitch Type", "RelSide", "HB", "plate_x", "plate_z"], axis = 1)

In [74]:
for i in range(len(league)):
    pitch_type = league.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        league.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        league.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        league.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        league.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO" or pitch_type == "FS":
        league.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        league.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        league.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        league.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [75]:
leaguePitch = league.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    HB = ('HB', 'mean'),
    RelHeight = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index()

leaguePitch = leaguePitch[leaguePitch['Pitches'] >= 100]
leaguePitch

,Name,Pitch Type,Pitches,Velo,Max_Velo,SpinRate,IVB,HB,RelHeight,Extension,Stuff
1,Aaron Bummer,FC,115,87.17,90.3,2343.65,0.8,-2.97,5.22,6.46,154.70
3,Aaron Bummer,SI,498,94.33,97.3,1972.83,1.99,14.51,5.09,6.51,108.83
4,Aaron Bummer,ST,418,82.53,86.1,2570.58,-2.6,-16.6,5.28,6.48,100.37
5,Aaron Civale,CU,486,78.1,81.2,2984.9,-15.98,10.88,6.44,5.64,131.68
6,Aaron Civale,FC,751,87.67,91.2,2595.9,7.67,3.75,6.38,5.68,122.87
...,...,...,...,...,...,...,...,...,...,...,...
2218,Zack Littell,FF,414,94.08,96.6,2069.47,17.35,-8.31,6.12,6.35,98.77
2219,Zack Littell,FS,254,84.26,86.8,914.18,2.25,-8.81,6.29,6.14,93.09
2220,Zack Littell,SI,127,92.88,94.9,2073.61,12.92,-15.14,6.06,6.29,100.13
2221,Zack Littell,SL,422,88.67,91.6,2211.72,7.82,0.75,6.18,6.24,105.51


In [76]:
leagueName = league.groupby(['Name']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    HB = ('HB', 'mean'),
    RelHeight = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index()

leagueName = leagueName[leagueName['Pitches'] >= 100]
leagueName

,Name,Pitches,Velo,Max_Velo,SpinRate,IVB,HB,RelHeight,Extension,Stuff
0,Aaron Bummer,1098,89.08,97.3,2258.04,0.64,0.64,5.18,6.5,109.55
1,Aaron Civale,2002,86.4,94.9,2610.25,3.92,1.22,6.35,5.68,118.96
2,Aaron Loup,832,87.33,94.6,2199.07,2.87,12.83,4.47,6.11,118.26
3,Adam Cimber,364,82.23,87.9,2234.35,3.07,-5.5,2.02,6.58,131.40
4,Adam Oller,412,88.83,96.0,2136.36,7.47,-0.87,5.79,6.29,99.51
...,...,...,...,...,...,...,...,...,...,...
534,Zack Greinke,2212,84.29,93.3,2239.11,5.65,-0.46,6.21,5.97,87.28
535,Zack Kelly,175,88.49,96.9,2276.88,9.67,-11.19,5.5,6.93,138.76
536,Zack Littell,1316,89.32,96.6,1903.11,9.51,-4.57,6.15,6.26,97.61
537,Zack Short,100,62.47,76.7,1482.49,17.93,-3.95,6.62,4.28,67.96


In [77]:
leagueNameStart = league.groupby(['Name']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    HB = ('HB', 'mean'),
    RelHeight = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2).reset_index()

leagueNameStart = leagueNameStart[leagueNameStart['Pitches'] >= 1300]
leagueNameStart

,Name,Pitches,Velo,Max_Velo,SpinRate,IVB,HB,RelHeight,Extension,Stuff
1,Aaron Civale,2002,86.4,94.9,2610.25,3.92,1.22,6.35,5.68,118.96
8,Alec Marsh,1342,89.1,97.4,2348.78,5.59,-3.85,5.67,6.37,109.02
10,Alek Manoah,1663,88.32,95.8,2215.1,8.88,-4.31,5.79,6.78,109.90
11,Alex Cobb,2491,90.4,97.1,2004.89,1.28,-9.62,5.77,5.91,106.76
16,Alex Wood,1690,87.27,95.4,1910.08,3.68,9.93,4.76,6.25,125.47
...,...,...,...,...,...,...,...,...,...,...
523,Zac Gallen,3241,89.53,96.8,2234.65,7.49,-1.8,5.91,6.59,118.19
524,Zach Davies,1497,86.07,92.6,2106.17,8.69,-11.31,5.51,6.12,84.98
525,Zach Eflin,2574,87.0,95.6,2240.19,3.32,-2.25,5.92,6.62,104.54
534,Zack Greinke,2212,84.29,93.3,2239.11,5.65,-0.46,6.21,5.97,87.28


In [78]:
league.to_csv('~/Desktop/Random-Projects/MLB/MLB-App/ALpitches.csv', index=False)
leaguePitch.to_csv('~/Desktop/Random-Projects/MLB/MLB-App/ALleaguePitch.csv', index=False)
leagueName.to_csv('~/Desktop/Random-Projects/MLB/MLB-App/ALleagueName.csv', index=False)
leagueNameStart.to_csv('~/Desktop/Random-Projects/MLB/MLB-App/ALleagueNameStart.csv', index=False)